# 시맨틱 커널용 플러그인 만들기

## 1. 사전 준비

In [ ]:
// 시맨틱 커널 패키지 추가
#r "nuget:Microsoft.SemanticKernel"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core,*-*"

| **Variables** | **Description** | **Example or See Also** |
|:----------:|------------|----------|
|`YourDeploymentName`| 소유하고 있는 모델의 배포 명 | `my-gpt-35-turbo-16k` |
|`YourEndpoint`| 소유하고 있는 API의 엔드포인트 | `https://{YOUR_ENDPOINT}.openai.azure.com/` |
|`YourApiKey`| 소유하고 있는 API Key | `abcde12345abcde12345abcde12345ab` |
|`YourModelId`| 사용하려는 모델 명 | https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#model-summary-table-and-region-availability |

⚠️ 주의! 사용하려는 모델이 지원하는 지역으로 Azure OpenAI 리소스를 생성해야 합니다.
이 예제에서는 `East US` 지역 리소스를 사용합니다.

In [ ]:
var yourDeploymentName = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("OpenAI 배포 이름을 입력해주세요.").ConfigureAwait(false);
var yourEndpoint = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("OpenAI Endpoint를 입력해주세요.").ConfigureAwait(false);
var yourApiKey = await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("OpenAI API Key를 입력해주세요.").ConfigureAwait(false);

In [ ]:
using Microsoft.SemanticKernel;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    deploymentName: yourDeploymentName,
    endpoint: yourEndpoint,
    apiKey: yourApiKey.GetClearTextPassword());

var kernel = builder.Build();

## 2. 프롬프트 입력하기

In [ ]:
var prompt = "늦은 아침으로 괜찮은 계란하고 치즈 들어간 음식 다섯개만 추천해줘";
var result = await kernel.InvokePromptAsync(prompt).ConfigureAwait(false);

Console.WriteLine(result);

## 3. 플러그인이란?

시맨틱 커널 SDK에서 플러그인은 커널에서 실행할 수 있는 함수를 포함하는 클래스입니다. 플러그인 함수는 시맨틱 프롬프트나 네이티브 함수 코드에서 만들어질 수 있습니다. 플러그인을 사용하려면 이를 커널에 추가한 다음 `InvokeAsync` 메서드를 사용하여 원하는 함수를 호출합니다. 커널은 플러그인에 액세스하여 함수를 찾아 실행한 다음 결과를 반환합니다. 몇 가지 미리 정의된 플러그인을 살펴보겠습니다.

## 4. 기본 제공 플러그인

시맨틱 커널 SDK는 일반적인 작업을 위해 미리 정의된 플러그인이 있는 추가 패키지를 제공합니다. 이는 NuGet과 함께 설치할 수 있는 [Plugins.Core](https://www.nuget.org/packages/Microsoft.SemanticKernel.Plugins.Core) 패키지에서 사용할 수 있습니다. `dotnet add package Microsoft.SemanticKernel.Plugins.Core --version 1.30.0-alpha` 패키지에는 다음 플러그인이 포함되어 있습니다.

* `ConversationSummaryPlugin` - 대화 요약
* `FileIOPlugin` - 파일 시스템에 대한 읽기 및 쓰기
* `HttpPlugin` - HTTP 엔드포인트에 대한 요청
* `MathPlugin` - 수학 연산 수행
* `TextPlugin` - 텍스트 조작 수행
* `TimePlugin` - 시간 및 날짜 정보 가져오기
* `WaitPlugin` - 지정된 시간 동안 실행 일시 중지

코어 플러그인을 사용하려면 `AddFromType` 메서드를 사용하여 커널 빌더에 추가할 수 있습니다. 예를 들어 커널에 `TimePlugin`을 추가하려면 다음 코드를 사용할 수 있습니다.

In [ ]:
// 실험용 기능 오류처리 안하도록 선언
#pragma warning disable SKEXP0050

using Microsoft.SemanticKernel.Plugins.Core;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    yourDeploymentName,
    yourEndpoint,
    yourApiKey.GetClearTextPassword());

// 커널에 TimePlugin을 추가
builder.Plugins.AddFromType<TimePlugin>();
var kernel = builder.Build();

// DayOfWeek 함수를 호출하여 현재 요일을 가져옵니다.
var currentDay = await kernel.InvokeAsync("TimePlugin", "DayOfWeek").ConfigureAwait(false);

Console.WriteLine(currentDay);

이번에는 `ConversationSummaryPlugin`을 이용해 대화 형식으로 사용해봅니다.

In [ ]:
// 실험용 기능 오류처리 안하도록 선언
#pragma warning disable SKEXP0050

using Microsoft.SemanticKernel.Plugins.Core;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    yourDeploymentName,
    yourEndpoint,
    yourApiKey.GetClearTextPassword());

// 커널에 ConversationSummaryPlugin을 추가
builder.Plugins.AddFromType<ConversationSummaryPlugin>();
var kernel = builder.Build();

// 대화 형식으로 답변을 가져옵니다.
string input = @"저는 새로운 레시피를 찾는 비건 채식주의자입니다. 저는 매운 음식을 좋아해요!
비건 친화적인 아침 식사 레시피 목록을 한글로 알려주실 수 있나요?";

var result = await kernel.InvokeAsync(
    "ConversationSummaryPlugin",
    "GetConversationActionItems",
    new() {{ "input", input }});

Console.WriteLine(result);

## 5. 프롬프트 작성 팁

* **특정 출력을 생성하는 특정 입력**
  * LLM은 수신하는 입력에 따라 응답합니다. 원하는 출력을 얻으려면 명확하고 구체적인 프롬프트를 만드는 것이 중요합니다.
* **다양하고 반복적인 테스트**
  * 모델이 응답을 해석하고 생성하는 방법을 이해하려면 다양한 프롬프트를 반복하고 실험해야 할 수 있습니다. 작은 조정으로 인해 결과가 크게 변할 수 있습니다.
* **컨텍스트의 중요성**
  * LLM은 프롬프트에 제공된 컨텍스트를 고려합니다. 정확하고 일관된 응답을 얻기 위해 컨텍스트가 잘 정의되고 관련성이 있는지 확인해야 합니다.
* **모호성 처리**
  * LLM은 모호한 쿼리로 어려움을 겪을 수 있습니다. 모호하거나 예기치 않은 결과를 방지하기 위해 컨텍스트 또는 구조를 제공합니다.
* **프롬프트 길이**
  * LLM은 짧고 긴 프롬프트를 모두 처리할 수 있지만 간결성과 명확성 간의 균형을 고려해야 합니다. 프롬프트 길이를 실험하면 최적의 균형을 찾는 데 도움이 될 수 있습니다.

## 6. 프롬프트 템플릿 만들기

시맨틱 커널 SDK는 자연어 프롬프트에서 식과 변수를 사용할 수 있는 템플릿 언어를 지원합니다. 이는 다양한 입력 매개 변수로 재사용 가능한 프롬프트를 만들 수 있음을 의미합니다. 프롬프트에 식을 포함하기 위해 템플릿 언어는 중괄호 `{{...}}`를 사용합니다.

In [ ]:
// 실험용 기능 오류처리 안하도록 선언
#pragma warning disable SKEXP0050

using Microsoft.SemanticKernel.Plugins.Core;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    yourDeploymentName,
    yourEndpoint,
    yourApiKey.GetClearTextPassword());

builder.Plugins.AddFromType<ConversationSummaryPlugin>();
var kernel = builder.Build();

string history = @"""
    분주한 주방 한가운데서 저는 가족의 다양한 입맛을 만족시키고
    각자의 독특한 취향을 탐색하는 도전을 받아들였습니다.
    편식하는 아이와 알레르기가 있는 아이가 섞여 있어
    제 요리 여정은 수많은 채식 레시피를 탐색하는 데 집중되어 있습니다.

    제 아이 중 한 명은 녹색 채소를 싫어하는 까다로운 식성이고,
    다른 한 명은 땅콩 알레르기가 있어 식사 계획에 복잡함을 더합니다.
    창의력과 건강한 요리에 대한 열정으로 무장한 저는 까다로운 입맛을 만족시킬 뿐만 아니라
    따뜻하고 맛있는 식물성 요리를 발견하는 풍미 가득한 모험을 시작했습니다.
""";

string prompt = @"사용자의 배경에 대한 몇 가지 정보입니다:
    {{$history}}

    이 사용자의 배경을 감안하여 관련 레시피 목록을 알려주세요.";

var result = await kernel.InvokePromptAsync(prompt,
    new KernelArguments() {{ "history", history }});

Console.WriteLine(result);

이 예에서 `history` 변수는 프롬프트에서 참조되며 `$` 기호로 표시됩니다. 프롬프트가 호출되면 `history` 변수가 `KernelArguments` 사전에 제공된 실제 값으로 바뀝니다. 이를 통해 다양한 입력으로 동적으로 채워질 수 있는 프롬프트를 만들 수 있습니다.

## 7. 파일에 프롬프트 저장

이전 단원에서는 `kernel.InvokePromptAsync`를 호출하여 재사용 가능한 프롬프트를 만드는 방법을 알아보았습니다.

인라인 프롬프트를 만드는 것이 유용하지만 대규모 프로젝트의 경우 프롬프트를 별도의 파일로 구성하고 커널로 가져올 수 있습니다. 이는 기본 제공 플러그인이 사용되는 방식과 유사합니다. 사용자 고유의 프롬프트 플러그인을 만들려면 프롬프트에 대해 별도의 폴더를 만드는 것이 가장 좋습니다.

## 8. 시맨틱 플러그인을 만드는 방법

시맨틱 커널 SDK는 몇 가지 간단한 구문 규칙을 사용하여 프롬프트 템플릿 언어를 지원합니다. 코드를 작성하거나 외부 라이브러리를 가져올 필요가 없으며 중괄호 `{{...}}`를 사용하여 프롬프트에 식을 포함하기만 하면 됩니다.

시맨틱 플러그인을 만들려면 `skprompt.txt` 파일과 `config.json` 파일이라는 두 개의 파일이 포함된 폴더가 필요합니다. `skprompt.txt` 파일에는 지금까지 작성한 모든 프롬프트와 유사한 LLM(대규모 언어 모델)에 대한 프롬프트가 포함되어 있습니다. `config.json` 파일에는 프롬프트에 대한 구성 세부 정보가 포함되어 있습니다.

`config.json` 파일은 다음 매개 변수를 지원합니다.

* `type`: 프롬프트의 유형입니다. 일반적으로 채팅 완료 프롬프트 형식을 사용합니다.
* `description`: 프롬프트가 수행하는 작업에 대한 설명입니다. 이 설명은 커널에서 프롬프트를 자동으로 호출하는 데 사용할 수 있습니다.
* `input_variables`: 프롬프트 내에서 사용되는 변수를 정의합니다.
* `execution_settings`: 완성 모델에 대한 설정입니다. OpenAI 모델의 경우 이러한 설정에는 max_tokens 및 temperature 속성이 포함됩니다.

예를 들어, 음악 강사 에이전트를 만든다고 가정해 보겠습니다. 잠재적인 코드 진행에 추가할 코드를 제안하는 기능을 지원할 수 있습니다. 이 경우 사용자는 시작 코드를 제공하고 플러그인은 잘 맞는 코드를 권장합니다.

이 플러그인을 만들려면 먼저 프로젝트에 `Prompts` 폴더를 만든 다음 `RecommendFood`라는 하위 폴더를 만듭니다. 그런 다음 `RecommendFood` 폴더에 `skprompt.txt` 및 `config.json` 파일을 추가합니다.


### `skprompt.txt` 파일의 예:

```txt
당신은 정말 친절한 음식 추천 도우미입니다. 당신은 매우 도움이 되고 창의적이죠.
고객이 현재 자신이 처한 상황을 먼저 알려줄거에요:
{{$history}}

고객은 {{$foodCategory}} 스타일의 음식을 원합니다.
고객이 만족할 수 있는 음식 목록을 제공해주세요.
```

### `config.json` 파일의 예:

```json
{
  "schema": 1,
  "type": "completion",
  "description": "Recommends food list to the user based on history",
  "execution_settings": {
    "default": {
      "max_tokens": 1000,
      "temperature": 0
    }
  },
  "input_variables": [
    {
      "name": "history",
      "description": "Food history about the user",
      "required": true
    },
    {
      "name": "foodCategory",
      "description": "The food category by user hopeful",
      "required": true
    }
  ]
}
```

이 예제에서 `temperature`는 생성된 텍스트의 임의화 정도를 제어하는 매개 변수입니다. 값은 `0`에서 `2` 사이여야 합니다. `temperature`가 낮을수록 더 집중적이고 정확한 출력이 가능하고, `temperature`가 높을수록 더 다양하고 창의적인 출력이 가능합니다.

현재 모델에서 요청은 프롬프트와 완료 간에 공유되는 최대 4,097개의 토큰을 사용할 수 있습니다. 이는 프롬프트가 4,000개의 토큰인 경우 채팅 완료는 최대 97개의 토큰이 될 수 있음을 의미합니다. LLM 설명서에서 미세 조정 매개 변수에 대한 자세한 정보를 찾을 수 있습니다.

사용자 지정 시맨틱 커널 플러그인을 사용하려면 프롬프트 디렉터리를 커널로 가져오고 해당 폴더 이름으로 플러그인을 호출합니다.

In [ ]:
// 실험용 기능 오류처리 안하도록 선언
#pragma warning disable SKEXP0050

using Microsoft.SemanticKernel.Plugins.Core;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    yourDeploymentName,
    yourEndpoint,
    yourApiKey.GetClearTextPassword());

var kernel = builder.Build();

var plugins = kernel.CreatePluginFromPromptDirectory("Prompts");

string history = @"""
    오늘은 아침에 삶은 계란을 먹었어요.
    점심에는 따듯한 국밥을 먹었죠.
    간식으로 과자를 조금 먹긴 했지만 저는 따뜻한 국물 요리를 좋아해요.
    """;
string category = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("음식 종류를 알려주세요.").ConfigureAwait(false);

var result = await kernel.InvokeAsync(
    plugins["RecommendFood"],
    new()
    {
        { "history", history },
        { "foodCategory", category }
    });

Console.WriteLine(result);